Change our model to use a 5 × 5 kernel with kernel_size=5 passed to the nn.Conv2d constructor. <br>
a What impact does this change have on the number of parameters in the model?<br>
b Does the change improve or degrade overfitting?<br>
c Read https://pytorch.org/docs/stable/nn.html#conv2d.<br>
d Can you describe what kernel_size=(1,3) will do?<br>
e How does the model behave with such a kernel?<br>

In [2]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import collections

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.set_printoptions(edgeitems=2)
torch.manual_seed(123)

In [3]:
class_names = ['airplane','automobile','bird','cat','deer',
               'dog','frog','horse','ship','truck']

In [4]:
from torchvision import datasets, transforms
data_path = '../data-unversioned/p1ch6/'
cifar10 = datasets.CIFAR10(
    data_path, train=True, download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

100%|██████████| 170498071/170498071 [00:02<00:00, 60942676.11it/s]


Extracting ../data-unversioned/p1ch6/cifar-10-python.tar.gz to ../data-unversioned/p1ch6/


In [5]:
cifar10_val = datasets.CIFAR10(
    data_path, train=False, download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4915, 0.4823, 0.4468),
                             (0.2470, 0.2435, 0.2616))
    ]))

Files already downloaded and verified


In [6]:
label_map = {0: 0, 2: 1}
class_names = ['airplane', 'bird']
cifar2 = [(img, label_map[label])
          for img, label in cifar10
          if label in [0, 2]]
cifar2_val = [(img, label_map[label])
              for img, label in cifar10_val
              if label in [0, 2]]

Model

The formula given for calculating the output size (one dimension) of a convolution is (W−F+2P)/S+1

If you want to keep the output size same as the input size, you can equate (W−max(F,S)+2P)/S+1=W
Solving this, when S=1
 gives:

W−F+2P+1=W

2P=F−1

When S>1
, you could solve the amount of padding needed to keep the output size same as the input size:

2P=max(F,S)−S

https://stats.stackexchange.com/questions/297678/how-to-calculate-optimal-zero-padding-for-convolutional-neural-networks

In [30]:
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=5, padding=2)
        self.conv2 = nn.Conv2d(16, 8, kernel_size=5, padding=2)
        self.fc1 = nn.Linear(8 * 8 * 8, 32)
        self.fc2 = nn.Linear(32, 2)

    def forward(self, x):
        out = F.max_pool2d(torch.tanh(self.conv1(x)), 2)
        out = F.max_pool2d(torch.tanh(self.conv2(out)), 2)
        out = out.view(-1, 8 * 8 * 8)
        out = torch.tanh(self.fc1(out))
        out = self.fc2(out)
        return out

Number of Parameters

In [31]:
model = Net()

numel_list = [p.numel() for p in model.parameters()]
sum(numel_list), numel_list

(20906, [1200, 16, 3200, 8, 16384, 32, 64, 2])

Training Loop

In [32]:
import datetime  # <1>

def training_loop(n_epochs, optimizer, model, loss_fn, train_loader):
    for epoch in range(1, n_epochs + 1):  # <2>
        loss_train = 0.0
        for imgs, labels in train_loader:  # <3>

            outputs = model(imgs)  # <4>

            loss = loss_fn(outputs, labels)  # <5>

            optimizer.zero_grad()  # <6>

            loss.backward()  # <7>

            optimizer.step()  # <8>

            loss_train += loss.item()  # <9>

        if epoch == 1 or epoch % 10 == 0:
            print('{} Epoch {}, Training loss {}'.format(
                datetime.datetime.now(), epoch,
                loss_train / len(train_loader)))  # <10>

We use the Dataset from chapter 7; wrap it into a DataLoader; instantiate our network, an optimizer, and a loss function as before; and call our training loop.
 The substantial changes in our model from the last chapter are that now our
model is a custom subclass of nn.Module and that we’re using convolutions. Let’s run
training for 100 epochs while printing the loss. Depending on your hardware, this
may take 20 minutes or more to finish!

In [33]:
train_loader = torch.utils.data.DataLoader(cifar2, batch_size=64,
                                           shuffle=True)  # <1>

model = Net()  #  <2>
optimizer = optim.SGD(model.parameters(), lr=1e-2)  #  <3>
loss_fn = nn.CrossEntropyLoss()  #  <4>

training_loop(  # <5>
    n_epochs = 100,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader,
)

2024-04-26 17:51:44.178096 Epoch 1, Training loss 0.5639230434302311
2024-04-26 17:52:37.200346 Epoch 10, Training loss 0.3269234761880462
2024-04-26 17:53:35.923911 Epoch 20, Training loss 0.2844390675520441
2024-04-26 17:54:34.117760 Epoch 30, Training loss 0.2556565905072887
2024-04-26 17:55:33.628358 Epoch 40, Training loss 0.2317938423555368
2024-04-26 17:56:31.910825 Epoch 50, Training loss 0.2051310299118613
2024-04-26 17:57:30.688470 Epoch 60, Training loss 0.18354635097228797
2024-04-26 17:58:29.351167 Epoch 70, Training loss 0.1645348866464226
2024-04-26 17:59:28.720147 Epoch 80, Training loss 0.14300961151814004
2024-04-26 18:00:27.506357 Epoch 90, Training loss 0.12541566530515433
2024-04-26 18:01:26.332072 Epoch 100, Training loss 0.10656613130478343


Checking Training and Validation Loss to see the amount of overfitting

In [34]:
train_loader = torch.utils.data.DataLoader(cifar2, batch_size=64,
                                           shuffle=False)
val_loader = torch.utils.data.DataLoader(cifar2_val, batch_size=64,
                                         shuffle=False)

def validate(model, train_loader, val_loader):
    for name, loader in [("train", train_loader), ("val", val_loader)]:
        correct = 0
        total = 0

        with torch.no_grad():  # <1>
            for imgs, labels in loader:
                outputs = model(imgs)
                _, predicted = torch.max(outputs, dim=1) # <2>
                total += labels.shape[0]  # <3>
                correct += int((predicted == labels).sum())  # <4>

        print("Accuracy {}: {:.2f}".format(name , correct / total))

validate(model, train_loader, val_loader)

Accuracy train: 0.96
Accuracy val: 0.89


Kernel_size can be a tuple of two ints – in which case, the first int is used for the height dimension, and the second int for the width dimension

There is a slight increase in the training accuracy but no change in Validation accuracy when we change filter size from 3x3 to 5x5

With Kernel size (1,3) using padding = 'same' which pads the input so the output has the shape as the input. However, this mode doesn’t support any stride values other than 1.

In [11]:
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=(1,3), padding='same')
        self.conv2 = nn.Conv2d(16, 8, kernel_size=(1,3), padding='same')
        self.fc1 = nn.Linear(8 * 8 * 8, 32)
        self.fc2 = nn.Linear(32, 2)

    def forward(self, x):
        out = F.max_pool2d(torch.tanh(self.conv1(x)), 2)
        out = F.max_pool2d(torch.tanh(self.conv2(out)), 2)
        out = out.view(-1, 8 * 8 * 8)
        out = torch.tanh(self.fc1(out))
        out = self.fc2(out)
        return out

In [12]:
model = Net()

numel_list = [p.numel() for p in model.parameters()]
sum(numel_list), numel_list

(17034, [144, 16, 384, 8, 16384, 32, 64, 2])

In [13]:
import datetime  # <1>

def training_loop(n_epochs, optimizer, model, loss_fn, train_loader):
    for epoch in range(1, n_epochs + 1):  # <2>
        loss_train = 0.0
        for imgs, labels in train_loader:  # <3>

            outputs = model(imgs)  # <4>

            loss = loss_fn(outputs, labels)  # <5>

            optimizer.zero_grad()  # <6>

            loss.backward()  # <7>

            optimizer.step()  # <8>

            loss_train += loss.item()  # <9>

        if epoch == 1 or epoch % 10 == 0:
            print('{} Epoch {}, Training loss {}'.format(
                datetime.datetime.now(), epoch,
                loss_train / len(train_loader)))  # <10>

In [14]:
train_loader = torch.utils.data.DataLoader(cifar2, batch_size=64,
                                           shuffle=True)  # <1>

model = Net()  #  <2>
optimizer = optim.SGD(model.parameters(), lr=1e-2)  #  <3>
loss_fn = nn.CrossEntropyLoss()  #  <4>

training_loop(  # <5>
    n_epochs = 100,
    optimizer = optimizer,
    model = model,
    loss_fn = loss_fn,
    train_loader = train_loader,
)

2024-04-26 21:26:11.400002 Epoch 1, Training loss 0.5659798695023652
2024-04-26 21:26:50.290938 Epoch 10, Training loss 0.34299551254245125
2024-04-26 21:27:31.790345 Epoch 20, Training loss 0.3158297742817812
2024-04-26 21:28:13.024979 Epoch 30, Training loss 0.2944501834880015
2024-04-26 21:28:55.054121 Epoch 40, Training loss 0.27680306620658585
2024-04-26 21:29:36.148278 Epoch 50, Training loss 0.2629637216116972
2024-04-26 21:30:17.983434 Epoch 60, Training loss 0.24467909226941456
2024-04-26 21:30:58.740232 Epoch 70, Training loss 0.23126557146667676
2024-04-26 21:31:40.665242 Epoch 80, Training loss 0.21842472611149405
2024-04-26 21:32:22.553775 Epoch 90, Training loss 0.20714929903958254
2024-04-26 21:33:04.600887 Epoch 100, Training loss 0.19390393776973341


In [15]:
train_loader = torch.utils.data.DataLoader(cifar2, batch_size=64,
                                           shuffle=False)
val_loader = torch.utils.data.DataLoader(cifar2_val, batch_size=64,
                                         shuffle=False)

def validate(model, train_loader, val_loader):
    for name, loader in [("train", train_loader), ("val", val_loader)]:
        correct = 0
        total = 0

        with torch.no_grad():  # <1>
            for imgs, labels in loader:
                outputs = model(imgs)
                _, predicted = torch.max(outputs, dim=1) # <2>
                total += labels.shape[0]  # <3>
                correct += int((predicted == labels).sum())  # <4>

        print("Accuracy {}: {:.2f}".format(name , correct / total))

validate(model, train_loader, val_loader)

Accuracy train: 0.92
Accuracy val: 0.89
